In [2]:
import sympy as sp
import numpy as np
from collections import OrderedDict
import tensorflow as tf
from my_util import show_graph, reset_graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9  
import copy
import time

def random_dist(n):
    '''A method for randomly sampling a distribution over n+1 states. Only the n probabilities are returned.'''
    a = sorted([0.0] + list(np.random.random_sample(n)))
    a = np.array([a[i + 1] - a[i] for i in range(n)], dtype = np.float64)    
    return a

Below is a vertex class that deals with vertex specific data and methods.

In [3]:
class Vertex():
    def __init__(self, name):
        self.vname = name
        self.parents = []
        self.hidden = False
        self.states = 0
        self.state = None
        self.dist = {}
        self._set_dist = {}
        self.trans = {}
        self._set_trans = {}       
        
    def add_parent(self, parent):
        '''Add a parent in the DAG'''
        self.parents.append(parent)
        
    def init_dist(self, train_thres):
        '''Initialize distribution for a vertex without parents with uniform distribution over the states. Number of states minus 1 tf variables are defined for the probabilities, where the last probability is determined by the rest. A tf operation for modifying the distribution is constructed.
        '''
        atom = np.float64(sp.sympify('1/'+str(self.states)))
        temp = []
        for state in range(self.states - 1):
            self.dist[state] = tf.Variable(atom, name = self.vname, dtype = tf.float64, trainable = self.hidden or (np.random.random()<train_thres))
            temp.append(self.dist[state])
            self._set_dist[state] = tf.assign(self.dist[state], X, validate_shape = True)

        self.dist[self.states - 1] = 1 - tf.add_n(temp)
    
    def pa_states(self):
        '''Return the parents' states'''
        temp = []
        for v in self.parents:
            temp.append(v.state)
        return temp
    
    def enum_pa_states(self, lev):
        '''Define a recursive generator for all combinations of parents' states'''
        if lev == len(self.parents):
            yield []
        else:    
            for i in range(self.parents[lev].states):
                for j in self.enum_pa_states(lev + 1):
                    yield [i] + j
                
    
    def init_trans(self, train_thres):
        '''Initialize conditional probabilities for a vertex with parents with uniform distribution over the states. Per combination of parents' states, number of states minus 1 tf variables are defined for the probabilities, where the last probability is determined by the rest, and a tf operation for modifying the conditional probabilities is constructed.
        '''
        atom = np.array(sp.sympify('1/'+str(self.states)), dtype = np.float64)
        for comb in self.enum_pa_states(0):
            temp = []
            for state in range(self.states - 1):
                tup = tuple(comb + [state])
                self.trans[tup] = tf.Variable(atom, name = self.vname, dtype = tf.float64, trainable = self.hidden or (np.random.random()<train_thres))
                temp.append(self.trans[tup])
                self._set_trans[tup] = tf.assign(self.trans[tup], X, validate_shape = True)
            tup = tuple(comb + [self.states - 1])
            self.trans[tup] = 1 - tf.add_n(temp)
        
    def set_dist(self, dist):
        '''Run the operation of modifying distributions in a tf session'''
        for state in range(self.states - 1):
            self._set_dist[state].eval(feed_dict = {X : dist[state]})
    
    def set_trans(self, comb, prob):
        '''Run the operation of modifying conditional probabilities in a tf session'''
        for state in range(self.states - 1):
            self._set_trans[tuple(comb + [state])].eval(feed_dict = {X : prob[state]})

Following is the main class for the Bayesian Network.

In [191]:
class BayesianNetwork():
    def __init__(self, filename = None, train_thres = 1.0):
        '''The input file has the format:
        Line 1: Vertex names separated by commas;
        Line 2: Indicator of hidden ('h') or observable ('o') for each vertex in order;
        Line 3-: One line for each vertex starting with name and comma,then either 'Root' if it has no parents or comma-separated list of parents.
        
        The train_thres is a parameter for experiment. Variables associated to non-root vertices have a probabilty train_thres to be trainable. By default, all parameters are trainable.
        '''
        self.vlist = OrderedDict()
        self.marg = {}
        if filename != None:
            f = open(filename, 'r')
            line = f.readline()
            items = line.split(',')
            for i in items:
                v = i.strip()
                self.vlist[v] = Vertex(v)
                
            line = f.readline()
            items = line.split(',')
            for (i, v), j in zip(self.vlist.items(), items):
                s = int(j)
                v.states = s
                
            line = f.readline()
            items = line.split(',')
            for (i, v), j in zip(self.vlist.items(), items):
                s = (j == 'h')
                v.hidden = s   
                
            line = f.readline()            
            while line != '':
                items = line.split(',')
                ch = self.vlist[items[0].strip()]
                pa = items[1].strip()
                trainable = ch.hidden
                if pa != 'Root': 
                    for i in range(1, len(items)):
                        ch.add_parent(self.vlist[items[i].strip()])
                    ch.init_trans(train_thres = train_thres)    
                else:
                    ch.init_dist(train_thres = train_thres)
                line = f.readline()
            f.close()
        self._vlist = list(self.vlist)
    
    def set_hidden(self, hidden):
        '''A helper method for setting exactly the vertex in 'hidden' as hidden vertices.
        '''
        for i, v in self.vlist.items():
            v.hidden = False
        for i in hidden:
            self.vlist[i].hidden = True

    def read_prob(self, filename):
        '''Read the vertex-wise distributions and conditional probabilities from a file.
        Each line starts with the vertex name, a combination of all parents' states if exist any, then a list of number of states - 1 numbers indicating the probabilities. See the accompanying input files for illustrations.
        The probabilities can be expressed as any mathematical expressions.
        
        e.g. For Allman et al. 2015 Table 1 Parameter set (2)
        
        a, 1/5
        b, 0, 4/5
        b, 1, 7/10
        c, 0,0,2/5
        c, 0,1,9/10
        c, 1,0,4/5
        c, 1,1,3/5
        ...
        '''
        if filename != None:
            f = open(filename, 'r')                
            line = f.readline()            
            while line != '':
                items = line.split(',')
                ch = self.vlist[items[0].strip()]
                if len(ch.parents) == 0: 
                    ch.set_dist(sp.matrix2numpy(sp.Matrix(sp.sympify((items[1:]))), np.float64).T[0]) 
                else:
                    ch.set_trans(list(map(int, items[1:len(ch.parents)+1])), sp.matrix2numpy(sp.Matrix(sp.sympify(items[len(ch.parents)+1:])), np.float64).T[0])
                line = f.readline()
            f.close()
    
    def parse(self, filename):
        '''
        A helper method for parsing the probabilities file, used only in read_prob_interpolate below.
        '''
        content = []
        f = open(filename, 'r')                
        line = f.readline()            
        while line != '':
            items = line.split(',')
            ch = self.vlist[items[0].strip()]
            if len(ch.parents) == 0: 
                content.append((ch, sp.matrix2numpy(sp.Matrix(sp.sympify((items[1:]))), np.float64).T[0]))
            else:
                content.append((ch, list(map(int, items[1:len(ch.parents)+1])), sp.matrix2numpy(sp.Matrix(sp.sympify(items[len(ch.parents)+1:])), np.float64).T[0]))
            line = f.readline()
        f.close()
        return content
    
    def read_prob_interpolate(self, filename1, filename2, size = 100):
        '''
        Interpolate between two sets of parameters linearly.
        '''
        file1 = self.parse(filename1)
        file2 = self.parse(filename2)
        for i in range(size + 1):
            for v1, v2 in zip(file1, file2):
                if len(v1[0].parents) == 0:
                    v1[0].set_dist((v1[1]*i+v2[1]*(size - i))/size)
                else:
                    v1[0].set_trans(v1[1], (v1[2]*i+v2[2]*(size - i))/size)
            print(i, target_loss.eval())
        return
    
    def sample_prob(self):
        '''
        Randomly sample an entire set of underlying parameters.
        '''
        for _, v in self.vlist.items():
            if len(v.parents) == 0:
                v.set_dist(random_dist(v.states - 1))
            else:
                for comb in v.enum_pa_states(0):
                    v.set_trans(comb, random_dist(v.states - 1))
        return
    
    def output_prob(self, filename, reverse = False):
        '''
        Output the current probabilities in a tf session to a file in the same format as input files.
        If reverse is True, flip the hidden labels assuming they are binary.
        '''
        f = open(filename, 'w')
        for _, v in self.vlist.items():
            if len(v.parents) == 0:
                f.write(v.vname)
                for state in range(v.states - 1):
                    if v.hidden and reverse:
                        f.write(","+str(1 - v.dist[state].eval()))
                    else:
                        f.write(","+str(v.dist[state].eval()))
                f.write('\n')
            else:
                for comb in v.enum_pa_states(0):
                    f.write(v.vname)
                    comb2 = comb
                    if v.parents[0].hidden and reverse:
                        comb2 = copy.deepcopy(comb)
                        comb[0] = 1-comb[0]
                    for j in comb2:
                        f.write(","+str(j))
                    for state in range(v.states - 1):
                        f.write(','+str(v.trans[tuple(comb + [state])].eval()))
                    f.write('\n')
        f.close()    
        return
            
    def get_constraint_loss(self):
        '''
        Construct a loss function to constrain all underlying parameters to be legal probabilities.
        '''
        temp = []
        for _, v in self.vlist.items():
            if len(v.parents) == 0:
                for _, i in v.dist.items():
                    temp.append(-(tf.minimum(i, 0)))
                    temp.append((tf.maximum(i-1, 0)))
            else:
                for _, i in v.trans.items():
                    temp.append(-(tf.minimum(i, 0)))
                    temp.append((tf.maximum(i-1, 0)))
        return tf.add_n(temp)
    
    def get_target_loss(self, target):
        '''
        Construct a loss function for the distance between the current marginal joint distribution on the observable vertices and the target observable distribution
        '''
        temp = []
        for obs, prob in self.marg.items():
            temp += [tf.abs(prob - target[obs])]
        return tf.add_n(temp)
    
    def compute_prob(self):
        '''
        Compute the total probability of a single configuation.
        '''
        temp = []
        for _, v in self.vlist.items():
            if len(v.parents) == 0:
                temp.append(v.dist[v.state])
            else:
                temp.append(v.trans[tuple(v.pa_states()+[v.state])])
        return tf.reduce_prod(temp)
    
    def enum_hidden(self, vi, prob_list):
        '''
        Given visible states, enumerate all hidden states.
        Call the method with vi = 0.
        prob_list is a callback list for storing the total probabilities.
        '''
        if vi == len(self.vlist):    
            prob_list.append(self.compute_prob())
            return
        else:   
            v = self.vlist[self._vlist[vi]]
            if v.hidden:
                for i in range(v.states):
                    v.state = i
                    self.enum_hidden(vi + 1, prob_list)
            else:
                self.enum_hidden(vi + 1, prob_list)
            return
                    
    
    def compute_observable(self, vi):
        '''
        Construct the tf operation for computing the marginal joint distributions over visible vertices.
        Call the method with vi = 0.
        '''
        if vi == len(self.vlist):
            obs = []
            for _, v in self.vlist.items():
                if not v.hidden:
                    obs.append(v.state)
            obs = tuple(obs)
            temp = []
            self.enum_hidden(0, temp)
            self.marg[obs] = tf.add_n(temp)    
            return
        else:
            v = self.vlist[self._vlist[vi]]
            if not v.hidden:
                for i in range(v.states):
                    v.state = i
                    self.compute_observable(vi + 1)
            else:
                self.compute_observable(vi + 1)
            return


First let us check the result using Allman et al. 2015 Table 1:

In [133]:
reset_graph(seed=int(time.time()))
X = tf.placeholder(tf.float64)
G = BayesianNetwork('graph 4-3e.txt', 1.0)
init = tf.global_variables_initializer()
G.compute_observable(0)
cons_loss = G.get_constraint_loss()

Using parameter set (1), we can compute the observation distributions. Here the states 0 and 1 correspond to states 1 and 2 in the paper. The tuple correponds to the states of vertices 1,2,3,4 in the paper in that order.

In [39]:
with tf.Session(config=config) as sess:
    init.run()
    G.read_prob('prob 4-3e a.txt')
    target_tb = sess.run(G.marg)
    for i, j in target_tb.items():
        print(i, j)

(0, 0, 0, 0) 0.1856
(0, 0, 0, 1) 0.05119999999999999
(0, 0, 1, 0) 0.2048
(0, 0, 1, 1) 0.07039999999999999
(0, 1, 0, 0) 0.05439999999999999
(0, 1, 0, 1) 0.020799999999999992
(0, 1, 1, 0) 0.0832
(0, 1, 1, 1) 0.0496
(1, 0, 0, 0) 0.05399999999999999
(1, 0, 0, 1) 0.0252
(1, 0, 1, 0) 0.0684
(1, 0, 1, 1) 0.032400000000000005
(1, 1, 0, 0) 0.0312
(1, 1, 0, 1) 0.013600000000000003
(1, 1, 1, 0) 0.0384
(1, 1, 1, 1) 0.016800000000000002


They look accurate except the last digit. We can rewrite them as fractions:

In [40]:
for i, j in target_tb.items():
    print(i, sp.Integer(sp.Expr.round(sp.sympify(j)*1000000,0))/1000000)

(0, 0, 0, 0) 116/625
(0, 0, 0, 1) 32/625
(0, 0, 1, 0) 128/625
(0, 0, 1, 1) 44/625
(0, 1, 0, 0) 34/625
(0, 1, 0, 1) 13/625
(0, 1, 1, 0) 52/625
(0, 1, 1, 1) 31/625
(1, 0, 0, 0) 27/500
(1, 0, 0, 1) 63/2500
(1, 0, 1, 0) 171/2500
(1, 0, 1, 1) 81/2500
(1, 1, 0, 0) 39/1250
(1, 1, 0, 1) 17/1250
(1, 1, 1, 0) 24/625
(1, 1, 1, 1) 21/1250


This exactly matches the tables at the bottom of Table 1 in the paper. Due to the way they are written in the paper, the order of reading their tables is the first element of each table, and then the second element of each table, etc.

We can check the observable distribution using parameter set (2), and verify that it is exactly the same as above.

In [41]:
with tf.Session(config=config) as sess:
    init.run()
    G.read_prob('prob 4-3e b.txt')
    target_tb = sess.run(G.marg)
    for i, j in target_tb.items():
        print(i, sp.Integer(sp.Expr.round(sp.sympify(j)*1000000,0))/1000000)

(0, 0, 0, 0) 116/625
(0, 0, 0, 1) 32/625
(0, 0, 1, 0) 128/625
(0, 0, 1, 1) 44/625
(0, 1, 0, 0) 34/625
(0, 1, 0, 1) 13/625
(0, 1, 1, 0) 52/625
(0, 1, 1, 1) 31/625
(1, 0, 0, 0) 27/500
(1, 0, 0, 1) 63/2500
(1, 0, 1, 0) 171/2500
(1, 0, 1, 1) 81/2500
(1, 1, 0, 0) 39/1250
(1, 1, 0, 1) 17/1250
(1, 1, 1, 0) 24/625
(1, 1, 1, 1) 21/1250


To check numerical stability, we run the gradient descent from a global minimum.

In [134]:
target_loss = G.get_target_loss(target_tb)
loss = target_loss + cons_loss
learning_rate = tf.placeholder(tf.float64)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
adamoptimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
adam_training_op = adamoptimizer.minimize(loss)
init2 = tf.global_variables_initializer()
saver = tf.train.Saver()

It turns out that Adadelta optimizer with a learning rate depending on the loss works better than other optimizers with adaptive learning rates, which often do not even converge starting from a global minimum.

In [119]:
with tf.Session(config=config) as sess:
    init2.run()
    G.read_prob('prob 4-3e a.txt')
    lr = 0.001
    counter = 0
    print('initial loss:', target_loss.eval())
    while True:
        for i in range(200):
            training_op.run(feed_dict = {learning_rate: lr})
        temp = target_loss.eval()  
        print(temp)
        lr = min(temp*10, lr)
        counter += 1
        if counter % 10 == 0:
            saver.save(sess, "Bayes/Save.ckpt")    
        if (temp < 1e-9) or (counter >= 10):
            break

initial loss: 5.898059818321144e-17
9.823684224939677e-07
1.4323991980383366e-08
5.363384510820302e-10


Compared to AdamOptimizer starting from the global minimum. Here the loss diverged so that the dynamic learning rate actually never kicked in. Changing initial and dynamic learning rates do not help either.

In [120]:
with tf.Session(config=config) as sess:
    init2.run()
    G.read_prob('prob 4-3e a.txt')
    lr = 0.001
    counter = 0
    print('initial loss:', target_loss.eval())
    while True:
        for i in range(200):
            adam_training_op.run(feed_dict = {learning_rate: lr})
        temp = target_loss.eval()  
        print(temp)
        lr = min(temp*10, lr)
        counter += 1
        if counter % 10 == 0:
            saver.save(sess, "Bayes/Save.ckpt")    
        if (temp < 1e-9) or (counter >= 10):
            break

initial loss: 5.898059818321144e-17
0.0005646576053426871
0.00048705921434175244
0.0006796705922605395
0.0002459418997772158
0.0005607448813293309
0.0005257245066200428
0.0005884374652615158
0.0003761820181770563
0.0003863891414351648
0.0004643808020423131


Now we modify the starting parameter slightly, by adding 0.1 to the single parameter on the hidden vertex.

In [136]:
with tf.Session(config=config) as sess:
    init2.run()
    G.read_prob('prob 4-3e a perturbed.txt')
    lr = 0.1
    counter = 0
    print('initial loss:', target_loss.eval())
    while True:
        for i in range(200):
            training_op.run(feed_dict = {learning_rate: lr})
        temp = target_loss.eval()  
        print(temp)
        lr = min(temp*10, lr)
        counter += 1
        if counter % 10 == 0:
            saver.save(sess, "Bayes/Save.ckpt")    
        if (temp < 1e-9) or (counter >= 20):
            break

initial loss: 0.12746666666666656
0.08576381252509549
0.03818969678199914
0.009597639408994727
0.006143380483190148
0.005446575377535087
0.005182456839372067
0.004870466710865081
0.004567360510793834
0.004371106241790497
0.004266719351624152
0.00417619932454827
0.004069039243926098
0.0039833619677277065
0.003876263341692743
0.0037817280431184912
0.0036758011220463013
0.003582163271944702
0.0034831825689687207
0.003411971838788148
0.003372190840244864


Here we changed the initial learning rate to 0.1 to speed up training, but dynamic learning rate kicked in later.

By running long enough, we see that the parameters no longer converge to the global minimum. I tried tuning the initial learning rate as well as the dynamic learning rate, but the same failure happens.

To see closer what is happening, let us make the hidden vertex the only trainable variable:

In [122]:
reset_graph(seed=int(time.time()))
X = tf.placeholder(tf.float64)
G = BayesianNetwork('graph 4-3e.txt', 0.0)
G.compute_observable(0)
cons_loss = G.get_constraint_loss()
target_loss = G.get_target_loss(target_tb)
loss = target_loss + cons_loss
learning_rate = tf.placeholder(tf.float64)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'a:0' shape=() dtype=float64_ref>]

In [123]:
with tf.Session(config=config) as sess:
    init.run()
    G.read_prob('prob 4-3e a perturbed.txt')
    lr = 0.1
    counter = 0
    while True:
        for i in range(200):
            training_op.run(feed_dict = {learning_rate: lr})
        temp = target_loss.eval()  
        print(temp)
        lr = min(temp*10, lr)
        counter += 1
        if counter % 10 == 0:
            saver.save(sess, "Bayes/Save.ckpt")    
        if (temp < 1e-9) or (counter >= 20):
            break

0.11415402224860025
0.09855516169729601
0.0809940668983128
0.0616696558894132
0.04073024667618916
0.01829198034509806
5.3746096042354974e-05
4.80493770178142e-07
4.994527744509036e-09
4.403401428210163e-11


We see that the single parameter did converge. If we allow a few more trainable parameters:

In [124]:
reset_graph(seed=int(time.time()))
X = tf.placeholder(tf.float64)
G = BayesianNetwork('graph 4-3e.txt', 0.15)
G.compute_observable(0)
cons_loss = G.get_constraint_loss()
target_loss = G.get_target_loss(target_tb)
loss = target_loss + cons_loss
learning_rate = tf.placeholder(tf.float64)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'a:0' shape=() dtype=float64_ref>,
 <tf.Variable 'c_3:0' shape=() dtype=float64_ref>,
 <tf.Variable 'd:0' shape=() dtype=float64_ref>,
 <tf.Variable 'e:0' shape=() dtype=float64_ref>]

In [126]:
with tf.Session(config=config) as sess:
    init.run()
    G.read_prob('prob 4-3e a perturbed.txt')
    lr = 0.1
    counter = 0
    while True:
        for i in range(200):
            training_op.run(feed_dict = {learning_rate: lr})
        temp = target_loss.eval()  
        print(temp)
        lr = min(temp*10, lr)
        counter += 1
        if counter % 10 == 0:
            saver.save(sess, "Bayes/Save.ckpt")    
        if (temp < 1e-9) or (counter >= 20):
            break

0.11241949110806096
0.094840966202152
0.07551785990049473
0.05845540731809814
0.04111639022832772
0.023462957569314914
0.004897656753087273
7.420591771698133e-05
1.0561646352665044e-06
6.253732807645629e-09
6.022641239888937e-11


It is still fine. Now we add more trainable variables:

In [137]:
reset_graph(seed=int(time.time()))
X = tf.placeholder(tf.float64)
G = BayesianNetwork('graph 4-3e.txt', 0.6)
G.compute_observable(0)
cons_loss = G.get_constraint_loss()
target_loss = G.get_target_loss(target_tb)
loss = target_loss + cons_loss
learning_rate = tf.placeholder(tf.float64)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'a:0' shape=() dtype=float64_ref>,
 <tf.Variable 'b:0' shape=() dtype=float64_ref>,
 <tf.Variable 'b_1:0' shape=() dtype=float64_ref>,
 <tf.Variable 'c_2:0' shape=() dtype=float64_ref>,
 <tf.Variable 'c_3:0' shape=() dtype=float64_ref>,
 <tf.Variable 'd:0' shape=() dtype=float64_ref>,
 <tf.Variable 'd_2:0' shape=() dtype=float64_ref>,
 <tf.Variable 'd_3:0' shape=() dtype=float64_ref>,
 <tf.Variable 'e_2:0' shape=() dtype=float64_ref>,
 <tf.Variable 'e_3:0' shape=() dtype=float64_ref>]

In [143]:
with tf.Session(config=config) as sess:
    init.run()
    G.read_prob('prob 4-3e a perturbed.txt')
    lr = 0.1
    counter = 0
    while True:
        for i in range(200):
            training_op.run(feed_dict = {learning_rate: lr})
        temp = target_loss.eval()  
        print(temp)
        lr = min(temp*10, lr)
        counter += 1
        if counter % 10 == 0:
            saver.save(sess, "Bayes/Save.ckpt")    
        if (temp < 1e-9) or (counter >= 100):
            break

0.09005131792381307
0.04793345191120958
0.02994913495511301
0.023225481843268815
0.018249773055080784
0.014905660869650525
0.012595204171079882
0.010835600997813987
0.009924203951723413
0.008980146438960775
0.008183128825176973
0.007451762227071602
0.006862895123938489
0.0062164867035253875
0.0056814281026376316
0.0051865477341188175
0.004820293368212632
0.004422461228555824
0.004122633816350016
0.0038235518553833264
0.0035106096263810074
0.0032545547263378664
0.0030418484122812436
0.002919115896453049
0.0027780285916682544
0.002668110670405132
0.0025614720476127197
0.0024589141607337904
0.00236284577186493
0.002267588244926776
0.002185237915973068
0.0020941658921485463
0.002015086103565737
0.0019277978531020545
0.0018533207196456455
0.001789740633898744
0.0017222134061891745
0.0016553106437521784
0.0015931224823813508
0.001532607684098325
0.0014798966821029918
0.0014220868399922818
0.0013636404721558883
0.0013200513675523402
0.0012623318966951593
0.0012144065904959624
0.00117325776121

By running long enough, this does not converge. So the high dimension caused problems by introducing either too many local minimums or saddle points.

Another question is that whether we can detect non-identifiability with infinite preimage. When the preimage is generically infinite, we might hope that it is much easier to find a global minimum after a perturbation from the initial parameter set, which would numerically verify non-identifiability. We choose graph 4-3h, as it is non-identifiable by a large margin. (25 to 15 dimensions)

In [196]:
reset_graph(seed=int(time.time()))
X = tf.placeholder(tf.float64)
G = BayesianNetwork('graph 4-3h.txt', 1.0)
init = tf.global_variables_initializer()
G.compute_observable(0)
cons_loss = G.get_constraint_loss()

We randomly generate two sets of parameters, where the second is obtained from the first one by flipping the binary label at the hidden vertex.

In [153]:
with tf.Session(config=config) as sess:
    init.run()
    G.sample_prob()
    G.output_prob('prob 4-3h a.txt')
    G.output_prob('prob 4-3h b.txt', reverse= True)
    target_tb = sess.run(G.marg)

In [197]:
target_loss = G.get_target_loss(target_tb)
loss = target_loss + cons_loss
learning_rate = tf.placeholder(tf.float64)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
init2 = tf.global_variables_initializer()
saver = tf.train.Saver()

We do the same perturbation the hidden vertex, by adding 0.1 to its single parameter.

In [189]:
with tf.Session(config=config) as sess:
    init2.run()
    G.read_prob('prob 4-3h a perturbed.txt')
    lr = 0.1
    counter = 0
    print('initial loss:', target_loss.eval())
    while True:
        for i in range(200):
            training_op.run(feed_dict = {learning_rate: lr})
        temp = target_loss.eval()  
        print(temp)
        lr = min(temp*10, lr)
        counter += 1
        if counter % 10 == 0:
            saver.save(sess, "Bayes/Save.ckpt")    
        if (temp < 1e-9) or (counter >= 20):
            break

initial loss: 0.1407614623564004
0.09958150071946262
0.05540704210828561
0.02682988070952828
0.021752939035413792
0.01764618985930428
0.015430257885244445
0.01376670145655901
0.012275381122024523
0.0114396740827586
0.01085492067780682
0.010327203492440528
0.00970841422455897
0.009206414890596893
0.008676613851552658
0.008218180010452582
0.00777462148451131
0.007377771936399928
0.006990199142466038
0.006610868759616301
0.006275474593536921


It eventually fails to find a global minimum as before. Here we could choose an even smaller perturbation, but it still does not converge to any global minimum.

Now again we allow a few trainable parameters:

In [283]:
reset_graph(seed=int(time.time()))
X = tf.placeholder(tf.float64)
G = BayesianNetwork('graph 4-3h.txt', 0.10)
G.compute_observable(0)
cons_loss = G.get_constraint_loss()
target_loss = G.get_target_loss(target_tb)
loss = target_loss + cons_loss
learning_rate = tf.placeholder(tf.float64)
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'a:0' shape=() dtype=float64_ref>,
 <tf.Variable 'b:0' shape=() dtype=float64_ref>,
 <tf.Variable 'b_1:0' shape=() dtype=float64_ref>,
 <tf.Variable 'e_10:0' shape=() dtype=float64_ref>]

In [284]:
with tf.Session(config=config) as sess:
    init.run()
    G.read_prob('prob 4-3h a perturbed.txt')
    lr = 0.1
    counter = 0
    while True:
        for i in range(200):
            training_op.run(feed_dict = {learning_rate: lr})
        temp = target_loss.eval()  
        print(temp)
        lr = min(temp*10, lr)
        counter += 1
        if counter % 10 == 0:
            saver.save(sess, "Bayes/Save.ckpt")    
        if (temp < 1e-9) or (counter >= 50):
            saver.save(sess, "Bayes/Save.ckpt")   
            break

0.1214741329302331
0.10168323430324643
0.07952008003835218
0.06321776885264489
0.04547685268410866
0.02631376166863068
0.005834017287473963
0.0001956507966063021
5.434260486788731e-06
7.1153557403697e-08
1.9447786801102263e-09
2.6235987340972322e-11


We can load up the checkpoint and compare the discovered parameters with the original parameters in 'prob 4-3h a perturbed.txt':

In [285]:
with tf.Session(config=config) as sess:
    saver.restore(sess, "Bayes/Save.ckpt")
    for i,v in G.vlist.items():
        if len(v.parents) == 0:
            for j, k in v.dist.items():
                print(v.vname, j,k.eval())
        else:
            for j, k in v.trans.items():
                print(v.vname, j,k.eval())

INFO:tensorflow:Restoring parameters from Bayes/Save.ckpt
a 0 0.34130945726920364
a 1 0.6586905427307963
b (0, 0) 0.5125407283592927
b (0, 1) 0.48745927164070735
b (1, 0) 0.8933298265933285
b (1, 1) 0.10667017340667151
c (0, 0) 0.08060593741515065
c (0, 1) 0.9193940625848493
c (1, 0) 0.6739409491924201
c (1, 1) 0.3260590508075799
d (0, 0) 0.6411257311081043
d (0, 1) 0.35887426889189566
d (1, 0) 0.2328541538614901
d (1, 1) 0.7671458461385099
e (0, 0, 0, 0, 0) 0.0792292225016149
e (0, 0, 0, 0, 1) 0.9207707774983851
e (0, 0, 0, 1, 0) 0.5150950739187724
e (0, 0, 0, 1, 1) 0.4849049260812276
e (0, 0, 1, 0, 0) 0.6346259144832795
e (0, 0, 1, 0, 1) 0.36537408551672046
e (0, 0, 1, 1, 0) 0.44030436942985174
e (0, 0, 1, 1, 1) 0.5596956305701483
e (0, 1, 0, 0, 0) 0.5912148034174216
e (0, 1, 0, 0, 1) 0.4087851965825784
e (0, 1, 0, 1, 0) 0.1697487039754937
e (0, 1, 0, 1, 1) 0.8302512960245063
e (0, 1, 1, 0, 0) 0.2202475397524286
e (0, 1, 1, 0, 1) 0.7797524602475714
e (0, 1, 1, 1, 0) 0.564866699691931

Up to numerical error, it is the same as the original parameters.

In summary, the failure to converge to a global minimum has made detecting non-identifiability with infinite preimage equally hard.

Finally, we examine the discrepency when we interpolate linearly between the two sets of parameters differing by flipping the hidden label.

In [190]:
with tf.Session(config=config) as sess:
    init2.run()
    G.read_prob_interpolate('prob 4-3h a.txt', 'prob 4-3h b.txt', 100)

0 1.0408340855860843e-17
1 0.017224183567053485
2 0.034186686674072105
3 0.050882226697932006
4 0.06730562993557461
5 0.08345183160400434
6 0.09931587584029214
7 0.11489291570157129
8 0.13017821316504058
9 0.14516713912796314
10 0.1598551734076662
11 0.17423790474154116
12 0.18831103078704475
13 0.20207035812169727
14 0.2155118022430836
15 0.22863138756885337
16 0.2414252474367199
17 0.2538896241044615
18 0.2660208687499208
19 0.27781544147100506
20 0.28926991128568524
21 0.30038095613199706
22 0.31114536286804106
23 0.32156002727198163
24 0.3316219540420474
25 0.34132825679653217
26 0.3506761580737935
27 0.3596629893322537
28 0.36828619095039933
29 0.37654331222678117
30 0.38443201138001487
31 0.39195005554877993
32 0.39909532079182086
33 0.4058657920879458
34 0.4122595633360281
35 0.418274837355005
36 0.42390992588387827
37 0.42916324958171403
38 0.4340333380276431
39 0.43851882972086026
40 0.4426184720806247
41 0.44633112144626064
42 0.4496557430771563
43 0.45259141115276363
44 0.45

So it is in fact a nice single peaked function. Even though the preimage is infinite in the parameter space, the submanifold of preimage does not seem to intersect the segment connecting these two points.